In [1]:
#Mounting the drive content that contains the necessary files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#installing the transformers package that is required for the utilization of the LEGAL-BERT
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.6 MB/s 
     |████████████████████████████████| 182 kB 65.4 MB/s 
     |████████████████████████████████| 7.6 MB 53.8 MB/s 


In [3]:
#importing the necessary modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import string
from nltk.corpus import stopwords
import numpy as np
import re
import tensorflow as tf
import tensorflow_hub as hub
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import os


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
#obtaining the list of stop words in english
stop_words = set(stopwords.words('english'))

#Defining the tokenizer, model and the unmasker that will perform the prediction
#Source: https://huggingface.co/nlpaueb/legal-bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

unmasker = pipeline('fill-mask', model="nlpaueb/legal-bert-base-uncased", tokenizer=tokenizer)


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertF

In [5]:
#function that cleans the lines from undesired characters as tags,punctuation signs etc.
def cleanline(line):
    tagpattern=r'(<)(.+?)(>)'
    numberpattern=r'([0-9]+)'
    cl=re.sub(tagpattern,'',line.lower()).replace('\n','').replace('“','').replace('”','').replace('…','').replace('—',' ').replace('‘','').replace('’','').replace('\\' , ' ').replace('/',' ')
    cl=re.sub(numberpattern,' ',cl)
    for char in cl:
            if (char in string.punctuation):
                cl=cl.replace(char,' ')
    return cl

In [6]:
#path of the file that contains the maxims to be augmented
TRAINSETPATHX='/content/drive/MyDrive/Maximsamples.txt'

f=open(TRAINSETPATHX,'r',encoding='utf8')
maximbits=f.readlines()
f.close()

#removing the endline characters
maximbits=[maxim[:-1] for maxim in maximbits]

print("The number of maxims: {}".format(len(maximbits)))

The number of maxims: 10


In [7]:
maximbits

['Lastly the rules which must make up the reference system should be identified according to objective criteria in particular to enable judicial review of the assessments on which that identification is based. It is for the Commission to take into account any factors put forward by the Member State concerned and more generally to carry out its examination in a rigorous and sufficiently reasoned manner in order to enable full judicial review.',
 'Second it is for the Commission when it approves a general aid scheme to take the necessary measures to ensure that the sectoral rules other than the general competition rules will be complied with by the Member State concerned.',
 'Having regard to the foregoing considerations the answer to be given to the first question must be that national measures which provide for a rebate of energy taxes on natural gas and electricity only in the case of undertakings whose activity is shown to consist primarily in the manufacture of goods must be regarde

In [8]:
#breaking the maxim lines into sentences
sentences=[]
nsentence=[] #storing the number of sentences each maxim contains
for line in maximbits:
    count=0
    #splitting sentences using ". " characters
    listsent=line.replace('\\\n','').split('. ')
    if(listsent==line.split('. ')):
      print("YES")
    else:
      print("NO")
    for i in range(0,len(listsent)):
        sentence=listsent[i]
        #sentences that are too short are eliminated
        if len(sentence)<5:
            continue
        elif (i==len(listsent)-1):
            #the last one is added as it is since we used the character sequence ". " for the seperation
            #and the last sentence contains only "."
            sentences.append(sentence)
            count=count+1
            continue
        else:
            #the "." characters are restored
            if(sentence[-1]!='.'):
                sentence=sentence+'.'
            sentences.append(sentence)
            count=count+1
    nsentence.append(count)

YES
YES
YES
YES
YES
YES
YES
YES
YES
YES


In [9]:
print(len(sentences))
print(len(nsentence))

14
10


In [10]:
perfix=[0.6] #percentage desired
noisesent6=[]   
#iterating over the sentences
for sent in sentences:
    print(sentences.index(sent))
    #tokinizing and capitalizing the sentences
    tokens = nltk.word_tokenize(cleanline(sent).capitalize())
    #lower-casing the tokens
    tokens=[token.lower() for token in tokens]
    #eliminating the stop words
    nonstoptoken = [word for word in tokens if not word in stop_words] 
    tags = nltk.pos_tag(nonstoptoken)
    #collecting the tokens with POS tag in categories nouns, adjectives and adverbs
    tochange=[tag for tag in tags if tag[1][0] in ['J','N','R']]
    nchange=[round(per*len(tochange)) for per in perfix] #number of required replacements due to the percentage given
    #print("Len to change: {}".format(len(tochange)))
    #print("Nchange: {}".format(nchange[0]))
    linemodtotal=sent.lower()
    indicator=sent[0].isupper()
    changed=0
    index=-1
    #adding the sentences into list if no change is supposed to be performed
    if nchange[0]==0:
        if (indicator):
          toadd=linemodtotal.replace('#','').capitalize()
        else:
          toadd=linemodtotal.replace('#','')
        noisesent6.append(toadd)
    i0=0
    while(changed<nchange[-1]):
        index+=1
        if(index<-len(tochange)):
            break
        word=tochange[index][0]
        #print("Word to change: {}".format(word))
        #The mask token [MASK] is placed in place of the word in process
        linemod=cleanline(sent).lower().replace(word,'[MASK]',1)
        #print(linemod)
        linemod=linemod+'.'
        #prediction has been made by the model in place of the [MASK] token
        filled=unmasker(linemod)
        if len(filled)>=1:
            for i in range(0,len(filled)):
                if(filled[i]['token_str']!=word):
                    toreplace=filled[i]['token_str']
                    #print("TOREPLACE: {}".format(toreplace))
                    changed+=1
                    #tracking the index in process to be able to replace multiple instances of words
                    #if the process just started
                    if i0==0:
                        i0=linemodtotal.find(word) 
                        pattern='(?<![a-zA-Z#])'+word+'(?![a-zA-Z#])'
                        linemodtotal=re.sub(pattern,'#'+toreplace+'#',linemodtotal,1)
                        break
                    #if the sentence is already processed before
                    else:
                        lm1=linemodtotal[0:i0]
                        lm2=linemodtotal[i0:]
                        i0=lm2.find(word)
                        pattern='(?<![a-zA-Z#])'+word+'(?![a-zA-Z#])'
                        lm2=re.sub(pattern,'#'+toreplace+'#',lm2,1)
                        linemodtotal=lm1+lm2
                        break
        else:
            continue
        #if the desired number of words are replaced
        if changed==nchange[0]:
            #indicator serves as a sign of the capitalized sentence
            #the "#" characters are removed at the end
            if (indicator):
              toadd=linemodtotal.replace('#','').capitalize()
            else:
              toadd=linemodtotal.replace('#','')
            #the augmented sentences added to the list defined at the beginning
            noisesent6.append(toadd)
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [11]:
#the path where the document containing the augmented samples will be saved to
FINALPATH="/content/"

noiselist=[noisesent6]
for element in noiselist:
    folder="0.6" #Could also bi iterated due to percentages. Here we use only one percentage value therefore it is strictly defined 
    final=[]
    start=0
    for i in range(0,len(nsentence)):
        toaddm=''
        n=nsentence[i]
        #merging the sentences that are contained in the same maxim
        for j in range(start,start+n):
            toaddm=toaddm+' '+element[j]
        toaddm=toaddm[1:]
        final.append(toaddm)
        start=start+n 
    #saving the augmented samples into a text document in the given final path
    f=open(FINALPATH+'/maximsamples_legalbert'+folder[-1]+'0.txt', 'w',encoding="utf8")
    for line in final:
        f.write(line+'\n')
    f.close()